In [1]:
!pip install -q -U google-generativeai langchain-google-genai chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.6 MB/s eta 0:00:

In [2]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [3]:
!sudo apt -y -qq install tesseract-ocr libtesseract-dev
!sudo apt-get -y-qq install poppler-utils libxm12-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig
!pip install langchain

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 49 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123634 files and directories

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [5]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

warnings.filterwarnings("ignore")


In [7]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/Attention-is-all-you-need-Paper.pdf")
data = loader.load()

In [8]:
len(data)

11

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in data)
text = text_splitter.split_text(context)

In [10]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [28]:
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
vector = embedding.embed_query("Attention is all you need.")

vector[:5]

[0.000157693779328838,
 -0.034076567739248276,
 -0.010322353802621365,
 -0.009999889880418777,
 0.0186016708612442]

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key= GEMINI_API_KEY,
    temperature= 0.2,
    convert_system_message_to_human=True,
    )

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GEMINI_API_KEY,

    )
vector_index = Chroma.from_texts(text, embeddings).as_retriever(search_kwargs={"k": 5})

In [18]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key= GEMINI_API_KEY,
    temperature= 0.2,
    convert_system_message_to_human=True,
    )

In [24]:
template = """
           Use the following pieces of context to answer the question in the end. If you don't know the answer, just say you didn't get the answer. {context}
           Question: {question}
           Helpful Answer
           """

In [25]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

In [27]:
question = "Define the multi head attention in detail"
result = qa_chain({"query":question})
result["result"]

'Multi-Head Attention involves linearly projecting the queries, keys, and values h times with different, learned linear projections to dk, dk, and dv dimensions, respectively. On each of these projected versions, the scaled dot-product attention function is performed in parallel, yielding dv-dimensional output values. These outputs are then concatenated and projected again, resulting in the final output. This allows the model to attend to information from different representation subspaces at different positions. The formula for Multi-Head Attention is:\n\nMultiHead(Q,K,V) = Concat(head1,..., headh)WO\n\nwhere headi = Attention(QWQi, KWKi, VWVi)\n\nHere, WQi ∈ Rdmodel×dk, WKi ∈ Rdmodel×dk, WVi ∈ Rdmodel×dv, and WO ∈ Rhdv×dmodel are the projection parameter matrices. In the paper, h = 8 parallel attention layers (heads) are used, with dk = dv = dmodel/h = 64.\n'